# Prepare Dataset

## Data Understanding
Images were obtained from two wildlife cameras (a Browning and Reconyx) set up near a beaver lodge on Lake Sammamish, Washington. Over 2,000 images were collected between April and June, 2019. Approximately 70% of the images contain beaver instances, the remaining are false triggers or other animals (roof rat, raccoon, squirrel, muskrat, otter, rabbit, frog, and various bird species). Most of the images are grayscale because the animals are more active at night. Both cameras were set to be motion activated and take at least 3 pictures when triggered. The date/time is off on the Browning camera and images are in weekly zipfiles with duplicate names across zipfiles. Challenges to object detection include imbalanced dataset (for non-beaver species), small instances (i.e. just tip of beaver tail), blur, grayscale, truncation, and occlusion.

## Data Cleaning

Image data is large, so before adding any data to my repo I made a data directory and added it to my .gitignore file. I used [Photos for macOS](https://www.apple.com/macos/photos/) and [ExifRenamer](https://www.macupdate.com/app/mac/10043/exifrenamer) to correct datetimes and rename all images by site, location, and datetime. There is only one site so far (s1). The Reconyx is camera 1 (c1), the Browning is camera 2 (c2). An example name for an image taken on the Reconyx on April 11th, 2019 at 9:30:56 a.m. looks like this s1c120190411_093056.jpg. Cleaned images were placed in a folder named data/images.

## Data Annotation

I used [LabelImg](https://github.com/tzutalin/labelImg) to make bounding boxes around animal instances in each image. If the image contained no animal instances I verified the image to create an annotation file with no bounding boxes. LabelImg creates a .xml file for each image. Annotations were generated in the folder data/annots. 

## Train Test Split Data

I decided to make two sets of data. A subset for testing my model pipeline and a dataset inclusive of all the data. I used 254 images of only beaver instances as my subset. 